In [ ]:
import cv2
import numpy as np
import glob
from scipy.spatial import distance
from imutils import face_utils
from keras.models import load_model
import tensorflow as tf
import requests
from fr_utils import *
from inception_blocks_v2 import *
import pymongo
from pymongo import MongoClient
from pprint import pprint
import urllib.parse
client = pymongo.MongoClient("place your mongo clint link that is found from mongodb atlas")
db=client["LocoTrackKLU"] #name of database
col1=db['students'] #name of collection
col2=db['teachers'] #name of collection
col3=db['managers'] #name of collection

In [ ]:
FR_model = load_model('nn4.small2.v1.h5') #name of the saved model 
#print("Total Params:", FR_model.count_params())

face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

threshold = 0.25

face_database = {}

In [ ]:
for name in os.listdir('images'):
    for image in os.listdir(os.path.join('images',name)):
        identity = os.path.splitext(os.path.basename(image))[0]
        face_database[identity] = fr_utils.img_path_to_encoding(os.path.join('images',name,image), FR_model)

#print(face_database)


In [ ]:
video_capture = cv2.VideoCapture(0)
ct=0
l=[]
while True:
    ret, frame = video_capture.read()
    frame = cv2.flip(frame, 1)
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    for(x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)
        roi = frame[y:y+h, x:x+w]
        encoding = img_to_encoding(roi, FR_model)
        min_dist = 100
        identity = None
        for(name, encoded_image_name) in face_database.items():
            dist = np.linalg.norm(encoding - encoded_image_name)
            if(dist < min_dist):
                min_dist = dist
                identity = name
#print('Min dist: ',min_dist)
        if min_dist < 0.1:
            ct+=1
            #cv2.putText(frame, "Face : " + identity[:-1], (x, y - 50), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
            cv2.putText(frame, "Dist : " + str(min_dist), (x, y - 20), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0),2)
            #print(identity[:-1].split("-")[0])
            IDD=identity[:-1].split("-")[0]
            l.append(IDD)
            myurl = 'https://ltklu.herokuapp.com/msf/uplocation' #update location of data that's hosted
            if ct%10==0:
                x=max(set(l),key=l.count)
                print(x)
                #rs = requests.patch(myurl, data={'id':x,'location':'ExpoGlug'})
                col1.update({'id':x}, {"$set":{'location':'ExpoGlug'}})
                col2.update({'id':x}, {"$set":{'location':'ExpoGlug'}})
                col3.update({'id':x}, {"$set":{'location':'ExpoGlug'}})
                #print(res.text)
        else:
            cv2.putText(frame, 'No matching faces', (x, y - 20), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)
    cv2.imshow('Face Recognition System', frame)
    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break
video_capture.release()
cv2.destroyAllWindows()